
# WeatherWise: Intelligent Weather Analysis & Advisory System

**Author:** _Your Name_  
**Environment:** Google Colab (Python 3)  
**Data Source:** `wttr.in` (no API key)  
**Libraries:** `requests`, `matplotlib`, `pyinputplus`, `colorama`, `re`

This notebook implements the **WeatherWise** application with:
- Live weather retrieval
- Natural language Q&A (rain/temperature/wind)
- Two visualisations (temperature & precipitation)
- A console menu UI (meets the assignment's `pyinputplus` requirement)

Use the **table of contents** to navigate sections. Run the notebook top-to-bottom.


## 1. Setup and Configuration

In [ ]:

# If running in a fresh Colab session, install dependencies once:
!pip -q install requests matplotlib pyinputplus colorama


In [ ]:

import os
import re
import requests
from urllib.parse import quote_plus
import matplotlib.pyplot as plt
import pyinputplus as pyip
from colorama import init as colorama_init, Fore, Style

# ---------- Global visual tuning (crisper, minimal) ----------
plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 120,
    "font.family": "sans-serif",
    "font.size": 9,
    "axes.titlesize": 10,
    "axes.labelsize": 9,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "axes.edgecolor": "#DDDDDD",
    "axes.linewidth": 0.6,
    "axes.labelcolor": "#444444",
    "text.color": "#333333",
})

# Init colorama once
colorama_init(autoreset=True)


## 2. Helper UI Utilities

In [ ]:

def clear_screen():
    os.system("cls" if os.name == "nt" else "clear")

def banner(text: str):
    line = "═" * (len(text) + 2)
    print(Fore.CYAN + f"╔{line}╗")
    print(Fore.CYAN + f"║ {Style.BRIGHT}{text}{Style.NORMAL} ║")
    print(Fore.CYAN + f"╚{line}╝")

def section(title: str):
    print("\n" + Fore.CYAN + Style.BRIGHT + f"► {title}")
    print(Fore.CYAN + "─" * (len(title) + 2))

def card_kv(label: str, value: str):
    print(Fore.WHITE + f"  {Style.BRIGHT}{label}:{Style.NORMAL} {value}")

def show_current_snapshot(weather_data: dict):
    \"\"\"Pretty mini-card with current conditions if available.\"\"\"
    cur = (weather_data or {}).get("current") or {}
    city = weather_data.get("resolved_area", "")
    if not cur:
        return
    section(f"Now in {city}")
    card_kv("Condition", cur.get("weatherDesc", "—"))
    card_kv("Temperature", f"{round(cur.get('tempC', 0))}°C")
    card_kv("Humidity", f"{round(cur.get('humidity', 0))}%")
    card_kv("Wind", f"{round(cur.get('windspeedKmph', 0))} km/h")


## 3. Weather Data Functions

In [ ]:

def get_weather_data(location: str, forecast_days: int = 5) -> dict:
    \"\"\"Retrieve weather data for a specified location using wttr.in (JSON).
    Handles incomplete data gracefully and normalises the response.
    \"\"\"
    if not isinstance(location, str) or not location.strip():
        return {"ok": False, "error": "location must be a non-empty string"}

    forecast_days = max(1, min(int(forecast_days), 5))
    encoded_location = quote_plus(location.strip())
    url = f"https://wttr.in/{encoded_location}?format=j1"

    try:
        resp = requests.get(url, timeout=15)
        resp.raise_for_status()
        data = resp.json()
    except requests.RequestException as e:
        return {"ok": False, "error": f"Network error: {e}"}
    except ValueError:
        return {"ok": False, "error": "Failed to parse JSON response"}

    try:
        current = (data.get("current_condition") or [{}])[0]
        weather = data.get("weather") or []
        days = weather[:forecast_days]

        # Resolve area name
        resolved_area = location
        near = data.get("nearest_area") or []
        if near and isinstance(near, list):
            first = near[0]
            if isinstance(first, dict):
                an = first.get("areaName") or []
                if an and isinstance(an, list) and "value" in an[0]:
                    resolved_area = an[0]["value"] or location

        # Build normalized structure
        norm_days = []
        for d in days:
            hourly = d.get("hourly") or []
            precip_prob = 0
            if hourly:
                try:
                    precip_prob = max(int(h.get("chanceofrain") or 0) for h in hourly)
                except Exception:
                    precip_prob = 0

            norm_days.append({
                "date": d.get("date"),
                "maxtempC": float(d.get("maxtempC") or 0),
                "mintempC": float(d.get("mintempC") or 0),
                "hourly": hourly,
                "precipProb": precip_prob,
            })

        return {
            "ok": True,
            "provider": "wttr.in",
            "resolved_area": resolved_area,
            "current": {
                "tempC": float(current.get("temp_C") or 0),
                "windspeedKmph": float(current.get("windspeedKmph") or 0),
                "humidity": float(current.get("humidity") or 0),
                "weatherDesc": ((current.get("weatherDesc") or [{"value": ""}])[0].get("value", "")),
            },
            "days": norm_days,
        }
    except Exception as e:
        return {"ok": False, "error": f"Unexpected error processing data: {e}"}


## 4. Visualisation Functions

In [ ]:

def create_temperature_visualisation(weather_data: dict, output_type: str = 'display'):
    \"\"\"Minimalist temperature chart with soft band, thin lines, clean grid.\"\"\"
    if not weather_data.get("ok"):
        raise ValueError(weather_data.get("error", "Bad weather data"))
    days = weather_data.get("days") or []
    if not days:
        raise ValueError("No forecast data to visualise")

    city = weather_data.get('resolved_area', '')
    x = [d["date"] for d in days]
    tmax = [float(d["maxtempC"]) for d in days]
    tmin = [float(d["mintempC"]) for d in days]

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.fill_between(x, tmin, tmax, color="#d0e6f7", alpha=0.4, linewidth=0)
    ax.plot(x, tmax, color="#0066cc", linewidth=1.8, marker="o", label="Max °C")
    ax.plot(x, tmin, color="#66a3e0", linewidth=1.8, marker="o", label="Min °C")

    ax.text(x[-1], tmax[-1], f"{round(tmax[-1])}°", color="#0066cc",
            fontsize=9, ha="left", va="bottom", weight="medium")
    ax.text(x[-1], tmin[-1], f"{round(tmin[-1])}°", color="#66a3e0",
            fontsize=9, ha="left", va="top", weight="medium")

    ax.set_title(f"Temperature Forecast — {city}", fontsize=10, loc="left", pad=8)
    ax.set_xlabel("Day"); ax.set_ylabel("Temperature (°C)")
    ax.grid(axis="y", color="#eaeaea", linewidth=0.5)
    for spine in ["top", "right"]:
        ax.spines[spine].set_visible(False)
    plt.xticks(rotation=20, ha="right", color="#555555")
    plt.yticks(color="#555555")
    ax.legend(frameon=False, loc="upper left", fontsize=8)
    plt.tight_layout()

    if output_type == 'figure':
        return fig
    plt.show()


def create_precipitation_visualisation(weather_data: dict, output_type: str = 'display'):
    \"\"\"Minimalist precipitation chart with clean bars and subtle grid.\"\"\"
    if not weather_data.get("ok"):
        raise ValueError(weather_data.get("error", "Bad weather data"))
    days = weather_data.get("days") or []
    if not days:
        raise ValueError("No forecast data to visualise")

    city = weather_data.get('resolved_area', '')
    x = [d["date"] for d in days]
    p = [int(d.get("precipProb", 0)) for d in days]

    fig, ax = plt.subplots(figsize=(8, 4))
    bars = ax.bar(x, p, color="#7db7e6", width=0.5)
    for rect, val in zip(bars, p):
        if val > 0:
            ax.text(rect.get_x() + rect.get_width()/2, rect.get_height() + 1,
                    f\"{val}%\", ha=\"center\", va=\"bottom\", color=\"#333333\", fontsize=8)

    ax.set_title(f\"Chance of Rain — {city}\", fontsize=10, loc=\"left\", pad=8)
    ax.set_xlabel(\"Day\"); ax.set_ylabel(\"Rain Probability (%)\")
    ax.set_ylim(0, 100)
    ax.grid(axis=\"y\", color=\"#eaeaea\", linewidth=0.5)
    for spine in [\"top\", \"right\"]:
        ax.spines[spine].set_visible(False)
    plt.xticks(rotation=20, ha=\"right\", color=\"#555555\")
    plt.yticks(color=\"#555555\")
    plt.tight_layout()

    if output_type == 'figure':
        return fig
    plt.show()


## 5. Natural Language Processing (Parsing & Response)

In [ ]:

def parse_weather_question(question: str) -> dict:
    \"\"\"Extract attribute, time window, and optional location from a plain-English question.\"\"\"
    q = (question or \"\").strip().lower()

    attr_map = {
        "rain": ["rain", "umbrella", "precip", "shower", "wet"],
        "temp": ["temp", "temperature", "hot", "cold", "warm", "cool", "heat"],
        "wind": ["wind", "gust", "breeze"],
    }
    attribute = "temp"
    for k, words in attr_map.items():
        if any(w in q for w in words):
            attribute = k
            break

    days_ahead = 0
    if "tomorrow" in q:
        days_ahead = 1
    elif "today" in q:
        days_ahead = 0
    else:
        m = re.search(r\"\\bin\\s+(\\d+)\\s+days?\\b\", q)
        if m:
            days_ahead = int(m.group(1))

    location = None
    m = re.search(r\"in\\s+([a-zA-Z\\-\\s]+)[?.!]*$\", (question or \"\").strip())
    if m:
        location = m.group(1).strip()

    return {\"attribute\": attribute, \"days_ahead\": days_ahead, \"location\": location}


def generate_weather_response(parsed_question: dict, weather_data: dict) -> str:
    \"\"\"Craft a natural-language answer from parsed intent + fetched data.\"\"\"
    city = weather_data.get(\"resolved_area\") or parsed_question.get(\"location\", \"that location\")

    if not weather_data.get(\"ok\"):
        return f\"Sorry—couldn't fetch weather data for {city}: {weather_data.get('error', 'unknown error')}.\"
    days = weather_data.get(\"days\") or []
    if not days:
        return f\"I couldn’t find any forecast data for {city}.\"

    idx = min(parsed_question.get(\"days_ahead\", 0), len(days) - 1)
    d = days[idx]
    attr = parsed_question.get(\"attribute\", \"temp\")

    if attr == \"rain\":
        prob = d.get(\"precipProb\", 0)
        hint = \"Take an umbrella.\" if prob >= 50 else \"Low chance of showers.\"
        return f\"In {city} on {d['date']}, the chance of rain is about {prob}%. {hint}\"

    elif attr == \"wind\":
        hourly = d.get(\"hourly\", [])
        max_wind = max([float(h.get(\"windspeedKmph\") or h.get(\"WindspeedKmph\") or 0) for h in hourly] or [0])
        return f\"In {city} on {d['date']}, peak wind speed will be around {round(max_wind)} km/h.\"

    max_temp = round(d.get(\"maxtempC\", 0))
    min_temp = round(d.get(\"mintempC\", 0))
    return f\"In {city} on {d['date']}, expect around {max_temp}°C max and {min_temp}°C min.\"


## 6. User Interface (Console Menu)

In [ ]:

def main_menu():
    \"\"\"Console UI for WeatherWise.\"\"\"
    clear_screen()
    banner(\"WeatherWise — Intelligent Weather Advisor\")

    while True:
        print()
        section(\"Main Menu\")
        choice = pyip.inputMenu(['Ask a question', 'Show charts', 'Quit'], numbered=True)

        if choice == 'Ask a question':
            clear_screen(); banner(\"Ask a Question\")
            q = pyip.inputStr(\"e.g., Will it rain tomorrow in Perth?\\n> \")
            parsed = parse_weather_question(q)
            location = parsed.get('location') or pyip.inputStr(\"Location (e.g., Perth): \")
            data = get_weather_data(location, forecast_days=5)

            section(\"Answer\")
            reply = generate_weather_response({**parsed, \"location\": location}, data)
            print(Fore.GREEN + reply)
            if data.get(\"ok\"): show_current_snapshot(data)
            print(\"\\n\" + Fore.CYAN + \"─\" * 60)

        elif choice == 'Show charts':
            clear_screen(); banner(\"Charts\")
            location = pyip.inputStr(\"Location (e.g., Perth): \")
            data = get_weather_data(location, forecast_days=5)
            if not data.get(\"ok\"):
                print(Fore.RED + f\"Error: {data.get('error')}\"); print(Fore.CYAN + \"─\" * 60); continue
            show_current_snapshot(data)
            section(\"Visualisations\")
            create_temperature_visualisation(data)
            create_precipitation_visualisation(data)
            print(Fore.CYAN + \"─\" * 60)

        else:
            print(\"\\n\" + Fore.YELLOW + \"Goodbye! Stay weather-wise. 🌦️\\n\")
            break


## 7. Main Application Logic

In [ ]:

# Uncomment the following line to run the menu interactively inside Colab's console:
# main_menu()


## 8. Testing and Examples

In [ ]:

# A. Smoke test (data fetch)
print("Fetch OK?:", get_weather_data("Perth", 3)["ok"])

# B. Q&A test
q = "Will it rain tomorrow in Perth?"
parsed = parse_weather_question(q)
data = get_weather_data(parsed.get("location") or "Perth", 3)
print("Q:", q)
print("A:", generate_weather_response(parsed, data))

# C. Charts test
data = get_weather_data("Perth", 3)
create_temperature_visualisation(data)
create_precipitation_visualisation(data)



## 9. Notes for Submission

- Keep this notebook in your GitHub repo alongside:
  - `README.md` (how to run + overview)
  - `conversation1.txt` … `conversation5.txt` (AI logs)
  - `reflection.docx` (300–500 words, Chicago 17th B author-date style)
- Exporting charts is optional; the grader will run cells to see outputs.
- If running `main_menu()` in Colab, use the console in the cell output to interact.
